LAUNCHING A NEW RESTAURANT AND EXTRACTING DATA AND INSIGHTS USING YELP'S GRAPHQL API

Code to Connect and Authenticate Yelp Graph QL API and Some Sample Data:

In [6]:
# First Import all required libraries to fetch and connect
import requests
import json

# Step 1: Fetch and enter the generated Yelp API Key (unique to each user)
api_key = '4bu_Cwth58wWv_1EItA96o4C7UXgyuq50aU8O6IgIu6fqW2fYPtwRYYpeyXReB5bUIDdm4XRIzOuacqB3JyvNYtjQ6JQ4ZLeB9LJ6EQ0U3ZOXJN2BDEycz1PqVo9Z3Yx'

# Step 2: Enter Yelp Graph QL API URL
url = 'https://api.yelp.com/v3/graphql'

# Step 3: Define the header and authorizations
headers = {
    'Authorization': f'Bearer {api_key}',
    'Content-Type': 'application/graphql',
}

# STEP 4: Define parameters/ data we want to pull
query = """
{
  business(id: "yelp-san-francisco") {
    name
    id
    alias
    rating
    url
    location {
      city
    }
  }
}
"""

# Step 5: Send POST Request with the GraphQL query as data

response = requests.post(url, headers=headers, data=query, timeout=10)

# Step 6: Check Response Status and Parse Data
if response.status_code == 200:
    data = response.json()
    business = data.get('data', {}).get('business', {})

    if business:
        print("Business Name:", business['name'])
        print("Business ID:", business['id'])
        print("Business Alias:", business['alias'])
        print("Rating:", business['rating'])
        print("URL:", business['url'])
        print("Location (City):", business['location']['city'])  # Updated print statement
    else:
            print("No business found.")
else:
    # Handle Rate Limit Exceeded
    if response.status_code == 429:
        print("Error: Rate limit exceeded. Please wait and try again later.")
        rate_limit_remaining = response.headers.get('RateLimit-Remaining', 'Unknown')
        rate_limit_reset = response.headers.get('RateLimit-Reset', 'Unknown')
        print(f"Rate Limit Remaining: {rate_limit_remaining}")
        print(f"Rate Limit Resets At: {rate_limit_reset}")
    else:
        print(f"Error: {response.status_code}")
        print("Response Text:", response.text)

Business Name: Yelp
Business ID: 4kMBvIEWPxWkWKFN__8SxQ
Business Alias: yelp-san-francisco
Rating: 2.0
URL: https://www.yelp.com/biz/yelp-san-francisco?adjust_creative=Ceoz5A7JyHPxGhjIOewJ3A&utm_campaign=yelp_api_v3&utm_medium=api_v3_graphql&utm_source=Ceoz5A7JyHPxGhjIOewJ3A
Location (City): San Francisco


Business Context and Some Sample Data in form of different queries from Graph QL

Query 1: Fetching Top 10 Korean Restaurants in Chicago City area limits

In [8]:
# 1.Define parameters/data for the query
query = """
{
  search(term: "Korean restaurant", location: "Chicago", limit: 10) {
    business {
      name
      rating
      location {
        address1
        city
        state
        country
      }
    }
  }
}
"""

# 2.Send POST Request with the GraphQL query as data
response = requests.post(url, headers=headers, data=query, timeout=10)

# 3. Check Response Status and Parse Data
if response.status_code == 200:
    data = response.json()
    businesses = data.get('data', {}).get('search', {}).get('business', [])

    if businesses:
        for i, business in enumerate(businesses, start=1):
            print(f"Restaurant {i}:")
            print(f"Name: {business['name']}")
            print(f"Rating: {business['rating']}")
            print(f"Address: {business['location']['address1']}, {business['location']['city']}, {business['location']['state']}, {business['location']['country']}")
            print("=" * 50)
    else:
        print("No businesses found.")
else:
    print(f"Error: {response.status_code}")

Restaurant 1:
Name: TAK Korean Bistro & Karaoke
Rating: 4.4
Address: 1132 S Wabash Ave, Chicago, IL, US
Restaurant 2:
Name: Daebak
Rating: 4.3
Address: 1266 N Milwaukee Ave, Chicago, IL, US
Restaurant 3:
Name: PERILLA fare
Rating: 4.1
Address: 401 N Milwaukee Ave, Chicago, IL, US
Restaurant 4:
Name: Cho Sun Ok Restaurant
Rating: 3.7
Address: 4200 N Lincoln Ave, Chicago, IL, US
Restaurant 5:
Name: Iron Age Korean Steakhouse
Rating: 3.9
Address: 1265 N Milwaukee Ave, Chicago, IL, US
Restaurant 6:
Name: Ban Po Jung
Rating: 4.1
Address: 3450 W Foster Ave, Chicago, IL, US
Restaurant 7:
Name: Ssyal
Rating: 4.3
Address: 4201 W Lawrence Ave, Chicago, IL, US
Restaurant 8:
Name: Han Bat Restaurant
Rating: 4.3
Address: 2723 W Lawrence Ave, Chicago, IL, US
Restaurant 9:
Name: Chungkiwa Restaurant
Rating: 4.7
Address: 1658 W Algonquin Rd, Mount Prospect, IL, US
Restaurant 10:
Name: Chicago Kalbi
Rating: 4.0
Address: 3752 W Lawrence Ave, Chicago, IL, US


There are really good amazing restuarants in Chicago area that cater needs of different cusines

Query 2: Lets get number of restaurant business units based on keyword Pizza in Chicago and its areas

In [9]:
# 1.Define parameters/data for the query
query = """
{
  search(term: "pizza", location: "Chicago", limit: 50) {
    total
    business {
      name
      location {
        city
      }
    }
  }
}
"""

# 2.Send POST Request with the GraphQL query as data
response = requests.post(url, headers=headers, data=query, timeout=10)

# 3. Check Response Status and Parse Data
if response.status_code == 200:
    data = response.json()
    businesses = data.get('data', {}).get('search', {}).get('business', [])

    if businesses:
        city_count = {}
        for business in businesses:
            city = business['location']['city']
            if city in city_count:
                city_count[city] += 1
            else:
                city_count[city] = 1

        # Display number of business units by cities
        for city, count in city_count.items():
            print(f"City: {city} | Number of Businesses: {count}")
    else:
        print("No businesses found.")
else:
    print(f"Error: {response.status_code}")


City: Chicago | Number of Businesses: 49
City: Evanston | Number of Businesses: 1


By this we can understand their are numerous Pizza serving restaurants in Chicago and its areas

Query 3: Getting Top 10 restaurants and their names based on most reviews

In [10]:
# 1.Define parameters/data for the query
query = """
{
  search(term: "restaurant", location: "Chicago", limit: 10) {
    business {
      name
      review_count
    }
  }
}
"""

# 2.Send POST Request with the GraphQL query as data
response = requests.post(url, headers=headers, data=query, timeout=10)

# 3. Check Response Status and Parse Data
if response.status_code == 200:
    data = response.json()
    businesses = data.get('data', {}).get('search', {}).get('business', [])

    if businesses:
        for business in businesses:
            print(f"Restaurant Name: {business['name']}")
            print(f"Review Count: {business['review_count']}")
            print("-" * 50)
    else:
        print("No businesses found.")
else:
    print(f"Error: {response.status_code}")


Restaurant Name: Girl & The Goat
Review Count: 10340
--------------------------------------------------
Restaurant Name: Cindy's Rooftop
Review Count: 2578
--------------------------------------------------
Restaurant Name: The Purple Pig
Review Count: 8698
--------------------------------------------------
Restaurant Name: Penumbra
Review Count: 953
--------------------------------------------------
Restaurant Name: The Dearborn
Review Count: 2448
--------------------------------------------------
Restaurant Name: No Vacancy
Review Count: 122
--------------------------------------------------
Restaurant Name: Ciccio Mio
Review Count: 519
--------------------------------------------------
Restaurant Name: Au Cheval
Review Count: 9120
--------------------------------------------------
Restaurant Name: Aba
Review Count: 1734
--------------------------------------------------
Restaurant Name: HIDE+SEEK
Review Count: 274
--------------------------------------------------


This helps us understand the most famous Restaurants based on customer reviews count and implement some strategies they are doing in our future business as well

Query 4: Now we will get Restaurants based on differnet cusine types they have to offer in Chicago area

In [11]:
# 1.Define parameters/data for the query
query = """
{
  search(term: "restaurant", location: "Chicago", limit: 10) {
    business {
      name
      location {
        address1
        city
        state
        country
      }
      url
      categories {
        title
      }
    }
  }
}
"""

# 2.Send POST Request with the GraphQL query as data
response = requests.post(url, headers=headers, data=query, timeout=10)

# 3. Check Response Status and Parse Data
if response.status_code == 200:
    data = response.json()
    businesses = data.get('data', {}).get('search', {}).get('business', [])

    if businesses:
        for business in businesses:
            # Extracting relevant data
            name = business.get('name', 'N/A')
            address = business.get('location', {}).get('address1', 'N/A')
            city = business.get('location', {}).get('city', 'N/A')
            state = business.get('location', {}).get('state', 'N/A')
            country = business.get('location', {}).get('country', 'N/A')
            url = business.get('url', 'N/A')
            categories = ", ".join([category['title'] for category in business.get('categories', [])])

            # Printing the results
            print(f"Restaurant Name: {name}")
            print(f"Address: {address}, {city}, {state}, {country}")
            print(f"URL: {url}")
            print(f"Categories (Cuisines): {categories}")
            print("-" * 50)
    else:
        print("No businesses found.")
else:
    print(f"Error: {response.status_code}")


Restaurant Name: Girl & The Goat
Address: 809 W Randolph, Chicago, IL, US
URL: https://www.yelp.com/biz/girl-and-the-goat-chicago?adjust_creative=Ceoz5A7JyHPxGhjIOewJ3A&utm_campaign=yelp_api_v3&utm_medium=api_v3_graphql&utm_source=Ceoz5A7JyHPxGhjIOewJ3A
Categories (Cuisines): New American, Bars, Bakeries
--------------------------------------------------
Restaurant Name: Cindy's Rooftop
Address: 12 S Michigan Ave, Chicago, IL, US
URL: https://www.yelp.com/biz/cindys-rooftop-chicago?adjust_creative=Ceoz5A7JyHPxGhjIOewJ3A&utm_campaign=yelp_api_v3&utm_medium=api_v3_graphql&utm_source=Ceoz5A7JyHPxGhjIOewJ3A
Categories (Cuisines): New American, Seafood, Breakfast & Brunch
--------------------------------------------------
Restaurant Name: The Purple Pig
Address: 444 N Michigan Ave, Chicago, IL, US
URL: https://www.yelp.com/biz/the-purple-pig-chicago?adjust_creative=Ceoz5A7JyHPxGhjIOewJ3A&utm_campaign=yelp_api_v3&utm_medium=api_v3_graphql&utm_source=Ceoz5A7JyHPxGhjIOewJ3A
Categories (Cuisine

This will give us stats and restaurants with different cusines, locations, websites they host and help us understand how each business unit varies and caters/ serves to customer needs

Query 5: Lets find some events happening in the Chicago area for Food servings or which have relation to Food in any way

In [12]:
import requests
import json

# Step 1: Fetch and enter the generated Yelp API Key (unique to each user)
api_key = '4bu_Cwth58wWv_1EItA96o4C7UXgyuq50aU8O6IgIu6fqW2fYPtwRYYpeyXReB5bUIDdm4XRIzOuacqB3JyvNYtjQ6JQ4ZLeB9LJ6EQ0U3ZOXJN2BDEycz1PqVo9Z3Yx'

# Step 2: Enter Yelp GraphQL API URL
url = 'https://api.yelp.com/v3/graphql'  # Ensure this is correct. Yelp mostly uses REST endpoints.

# Step 3: Define the header and authorizations
headers = {
    'Authorization': f'Bearer {api_key}',
    'Content-Type': 'application/json',  # Correct content type
}

# 1. Define parameters/data for the query
category = "Food"  # Setting the category to 'Food'
location = "Chicago"
query = {
    "query": f"""
    {{
      search(term: "{category}", location: "{location}") {{
        business {{
          name
          rating
          location {{
            address1
            city
            state
            country
          }}
          categories {{
            title
          }}
        }}
      }}
    }}
    """
}

# 2. Send POST Request with the GraphQL query as data
response = requests.post(url, headers=headers, json=query, timeout=10)

# 3. Check Response Status and Parse Data
if response.status_code == 200:
    data = response.json()
    businesses = data.get('data', {}).get('search', {}).get('business', [])

    if businesses:
        for business in businesses:
            # Extracting relevant data
            name = business.get('name', 'N/A')
            rating = business.get('rating', 'N/A')
            location = business.get('location', {})
            address = location.get('address1', 'N/A')
            city = location.get('city', 'N/A')
            state = location.get('state', 'N/A')
            country = location.get('country', 'N/A')
            categories = [category.get('title', 'N/A') for category in business.get('categories', [])]

            # Printing the business details
            print(f"Business Name: {name}")
            print(f"Rating: {rating}")
            print(f"Address: {address}, {city}, {state}, {country}")
            print(f"Categories: {', '.join(categories)}")
            print("-" * 50)
    else:
        print(f"No {category} businesses found in {location}.")
else:
    # Handle Rate Limit Exceeded
    if response.status_code == 429:
        print("Error: Rate limit exceeded. Please wait and try again later.")
        rate_limit_remaining = response.headers.get('RateLimit-Remaining', 'Unknown')
        rate_limit_reset = response.headers.get('RateLimit-Reset', 'Unknown')
        print(f"Rate Limit Remaining: {rate_limit_remaining}")
        print(f"Rate Limit Resets At: {rate_limit_reset}")
    else:
        print(f"Error: {response.status_code}")
        print("Response Text:", response.text)


Business Name: Way Out
Rating: 4.7
Address: 3213 W Armitage Ave, Chicago, IL, US
Categories: Chicken Wings, Cocktail Bars
--------------------------------------------------
Business Name: Tama
Rating: 4.6
Address: 1952 N Damen Ave, Chicago, IL, US
Categories: Greek, Bars
--------------------------------------------------
Business Name: Meez Meals
Rating: 4.8
Address: , Chicago, IL, US
Categories: Food Delivery Services
--------------------------------------------------
Business Name: Umaga Bakehouse
Rating: 4.3
Address: 4703 W Foster Ave, Chicago, IL, US
Categories: Bakeries, Filipino
--------------------------------------------------
Business Name: Aloha Eats
Rating: 4.1
Address: 2534 N Clark St, Chicago, IL, US
Categories: Hawaiian, Barbeque
--------------------------------------------------
Business Name: Velvet Taco
Rating: 4.2
Address: 1110 N State St, Chicago, IL, US
Categories: New American, Tacos, Vegetarian
--------------------------------------------------
Business Name: Ying

This will help us understand and analyze if their are events focused on Food as events are one such example were customers engage a lot and if more events happen in and around an area which means more customers and people are focused on those line of area

Query 6: Fetching reviews by different languages used by Customers

In [13]:
import requests
import json

# Step 1: Fetch and enter the generated Yelp API Key (unique to each user)
api_key = '4bu_Cwth58wWv_1EItA96o4C7UXgyuq50aU8O6IgIu6fqW2fYPtwRYYpeyXReB5bUIDdm4XRIzOuacqB3JyvNYtjQ6JQ4ZLeB9LJ6EQ0U3ZOXJN2BDEycz1PqVo9Z3Yx'

# Step 2: Enter Yelp GraphQL API URL
url = 'https://api.yelp.com/v3/graphql'

# Step 3: Define the header and authorizations
headers = {
    'Authorization': f'Bearer {api_key}',
    'Content-Type': 'application/json',  # Use 'application/json' for GraphQL
}

# 1. Define parameters/data for the query
query = """
{
  search(term: "restaurant", location: "Chicago", limit: 10) {
    business {
      id
      name
      rating
      reviews {
        id
        rating
        text
        user {
          name
        }
      }
    }
  }
}
"""

# 2. Send POST Request with the GraphQL query as data
response = requests.post(url, headers=headers, json={'query': query}, timeout=10)

# 3. Check Response Status and Parse Data
if response.status_code == 200:
    data = response.json()

    businesses = data.get('data', {}).get('search', {}).get('business', [])

    if businesses:
        for business in businesses:
            print(f"Business Name: {business['name']}")
            print(f"Rating: {business['rating']}")
            print("Reviews:")

            # Process the reviews for each business
            for review in business.get('reviews', []):
                print(f"  Review ID: {review['id']}")
                print(f"  Rating: {review['rating']}")
                print(f"  Text: {review['text']}")
                print(f"  User: {review['user']['name']}")
                print("-" * 50)
    else:
        print("No restaurants found in Chicago.")
else:
    print(f"Error: {response.status_code}")
    print("Response Text:", response.text)


Business Name: Girl & The Goat
Rating: 4.4
Reviews:
  Review ID: YMEqQ3phqgaSpBFBWx3LBg
  Rating: 5
  Text: Loved my dining experience here while visiting Chicago. We were recommended by friends who are from the area and we made a late reservation.

The vibes and...
  User: Leslie M.
--------------------------------------------------
  Review ID: 9L-00Oerpu9CGjMxkprGKQ
  Rating: 4
  Text: Dinner for 6.

- Ambiance (5*)
Sophisticated vibe. Cozy. Busy but not too loud. 

- Service (3*)
Very mixed bag with one major issue that overshadowed an...
  User: Jeff J.
--------------------------------------------------
  Review ID: KHq6QDlw_ZoCz-1oDGeA6w
  Rating: 4
  Text: We had a very nice chef table far right corner rear of restaurant, very cozy. 
Our server Jacob was very knowledgeable and sexy.
We had delicious empanadas,...
  User: Sean W.
--------------------------------------------------
Business Name: Cindy's Rooftop
Rating: 4.1
Reviews:
  Review ID: K6P5r55naKd5SVIpBaxP3w
  Rating: 5
 

This will give us idea on how customers use different languages to post their reviews and when we we open our restaurant then need to make sure we have language signs and guide cutomers and make them understand menu and our restaurant givings in different languages

Query 7:Here I am fetching top 10 customers and their details if available who have posted more than 100 reviews on Yelp.

In [16]:
import requests
import json
import time

# Step 1: Fetch and enter the generated Yelp API Key (unique to each user)
api_key = '4bu_Cwth58wWv_1EItA96o4C7UXgyuq50aU8O6IgIu6fqW2fYPtwRYYpeyXReB5bUIDdm4XRIzOuacqB3JyvNYtjQ6JQ4ZLeB9LJ6EQ0U3ZOXJN2BDEycz1PqVo9Z3Yx'

# Step 2: Enter Yelp GraphQL API URL
url = 'https://api.yelp.com/v3/graphql'

# Step 3: Define the header and authorizations
headers = {
    'Authorization': f'Bearer {api_key}',
    'Content-Type': 'application/json',  # Correct header for GraphQL query in JSON format
}

# 1. Define parameters/data for the query
query = """
{
  search(term: "business", location: "Chicago", limit: 10) {
    total
    business {
      id
      name
      rating
      reviews {
        id
        rating
        text
        user {
          name

        }
      }
    }
  }
}
"""

# 2. Send POST Request with the GraphQL query as data
response = requests.post(url, headers=headers, json={'query': query}, timeout=10)

# 3. Check Response Status and Parse Data
if response.status_code == 200:
    data = response.json()
    businesses = data.get('data', {}).get('search', {}).get('business', [])

    if businesses:
        for business in businesses:
            print(f"Business Name: {business['name']}")
            print(f"Rating: {business['rating']}")
            print("Reviews:")

            # Process the reviews for each business
            for review in business.get('reviews', []):
                print(f"  Review ID: {review['id']}")
                print(f"  Rating: {review['rating']}")
                print(f"  Text: {review['text']}")
                print(f"  User: {review['user']['name']}")
                print("-" * 50)
    else:
        print("No businesses found in Chicago.")
else:
    if response.status_code == 429:
        print(f"Rate Limit Reached: {response.status_code}")
        rate_limit_remaining = response.headers.get('X-RateLimit-Remaining', 'Unknown')
        rate_limit_reset = response.headers.get('X-RateLimit-Reset', 'Unknown')
        print(f"Rate Limit Remaining: {rate_limit_remaining}")
        print(f"Rate Limit Resets At: {rate_limit_reset}")

        # Convert reset time (assuming it comes as a Unix timestamp) to local time
        if rate_limit_reset != 'Unknown':
            reset_time = time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(int(rate_limit_reset)))
            print(f"Rate Limit Resets At: {reset_time} GMT")
    else:
        print(f"Error: {response.status_code}")
        print(response.text)


Business Name: Insanely Handy
Rating: 5.0
Reviews:
  Review ID: WaXWXyzM4madKuWfDXnG2g
  Rating: 5
  Text: Chris from Insanely Handy was amazing! I hired him for some jobs at my business. He installed a floor to ceiling curtain, a smart thermostat and changed the...
  User: C-Y L.
--------------------------------------------------
Business Name: O'Brien the Cleaner
Rating: 5.0
Reviews:
  Review ID: yhfFq4b8rbBCpcz8G0z-PA
  Rating: 5
  Text: Dan goes above and beyond for his clients. He knows his business and makes an effort to get to know you and your preferences with tailoring and cleaning. We...
  User: Gabriel P.
--------------------------------------------------
  Review ID: KXyKvYCeLmBR7660eprMpQ
  Rating: 5
  Text: Best service ever. Found out about this cleaner as my previous location I would visit was closed. I read the reviews and decided to give it a try. Not...
  User: Nereida H.
--------------------------------------------------
  Review ID: QO8V_vwMxd06hP27Vg1oIg
  Rating:

This means they are frequent users of Yelp and understanding and analyzing their patterns can be very helpful to setup my new business venture and understand what customers want.

Query 8: Getting customers by their Yelp Join Dates and Years

In [18]:
# First Import all required libraries to fetch and connect
import requests
import json

# Step 1: Fetch and enter the generated Yelp API Key (unique to each user)
api_key = '4bu_Cwth58wWv_1EItA96o4C7UXgyuq50aU8O6IgIu6fqW2fYPtwRYYpeyXReB5bUIDdm4XRIzOuacqB3JyvNYtjQ6JQ4ZLeB9LJ6EQ0U3ZOXJN2BDEycz1PqVo9Z3Yx'

# Step 2: Enter Yelp Graph QL API URL
url = 'https://api.yelp.com/v3/graphql'

# Step 3: Define the header and authorizations
headers = {
    'Authorization': f'Bearer {api_key}',
    'Content-Type': 'application/graphql',
}

# 1.Define parameters/data for the query
query = """
{
  search(term: "reviewer", location: "Chicago", limit: 50) {
    total
    }
  }
"""

# 2.Send POST Request with the GraphQL query as data
response = requests.post(url, headers=headers, data=query, timeout=10)

# 3. Check Response Status and Parse Data
if response.status_code == 200:
    data = response.json()
    users = data.get('data', {}).get('search', {}).get('user', [])

    if users:
        # Extract yelping_since (join year) and count occurrences
        join_years = [user['yelping_since'][:4] for user in users]  # Extract year (first 4 characters)
        year_counts = Counter(join_years)

        # Display the number of users joined each year
        for year, count in year_counts.items():
            print(f"Year: {year}, Users Joined: {count}")
    else:
        print("No users found.")
else:
    print(f"Error: {response.status_code}")
    print(response.text)


No users found.


This will give us some some estimate to how many customers Yelp engages and are its users

Query 9: Query to know Icecream Parlours in Chicago area

In [19]:
# First Import all required libraries to fetch and connect
import requests
import json

# Step 1: Fetch and enter the generated Yelp API Key (unique to each user)
api_key = '4bu_Cwth58wWv_1EItA96o4C7UXgyuq50aU8O6IgIu6fqW2fYPtwRYYpeyXReB5bUIDdm4XRIzOuacqB3JyvNYtjQ6JQ4ZLeB9LJ6EQ0U3ZOXJN2BDEycz1PqVo9Z3Yx'

# Step 2: Enter Yelp Graph QL API URL
url = 'https://api.yelp.com/v3/graphql'

# Step 3: Define the header and authorizations
headers = {
    'Authorization': f'Bearer {api_key}',
    'Content-Type': 'application/graphql',
}

# 1.Define parameters/data for the query
query = """
{
  search(term: "ice cream", location: "Chicago", limit: 10) {
    total
    business {
      name
      location {
        address1
        city
        state
      }
      url
    }
  }
}
"""

# 2.Send POST Request with the GraphQL query as data
response = requests.post(url, headers=headers, data=query, timeout=10)

# 3. Check Response Status and Parse Data
if response.status_code == 200:
    data = response.json()
    businesses = data.get('data', {}).get('search', {}).get('business', [])

    if businesses:
        print("Restaurants serving ice cream in Chicago:")
        for business in businesses:
            print(f"Name: {business['name']}")
            print(f"Address: {business['location']['address1']}, {business['location']['city']}, {business['location']['state']}")
            print(f"URL: {business['url']}")
            print("-" * 50)
    else:
        print("No ice cream-serving restaurants found.")
else:
    print(f"Error: {response.status_code}")


Restaurants serving ice cream in New York:
Name: Jeni's Splendid Ice Creams
Address: 1505 N Milwaukee Ave, Chicago, IL
URL: https://www.yelp.com/biz/jenis-splendid-ice-creams-chicago-3?adjust_creative=Ceoz5A7JyHPxGhjIOewJ3A&utm_campaign=yelp_api_v3&utm_medium=api_v3_graphql&utm_source=Ceoz5A7JyHPxGhjIOewJ3A
--------------------------------------------------
Name: Margie's Candies
Address: 1960 N Western Ave, Chicago, IL
URL: https://www.yelp.com/biz/margies-candies-chicago?adjust_creative=Ceoz5A7JyHPxGhjIOewJ3A&utm_campaign=yelp_api_v3&utm_medium=api_v3_graphql&utm_source=Ceoz5A7JyHPxGhjIOewJ3A
--------------------------------------------------
Name: Eli's Ark
Address: 1843 W North Ave, Chicago, IL
URL: https://www.yelp.com/biz/elis-ark-chicago?adjust_creative=Ceoz5A7JyHPxGhjIOewJ3A&utm_campaign=yelp_api_v3&utm_medium=api_v3_graphql&utm_source=Ceoz5A7JyHPxGhjIOewJ3A
--------------------------------------------------
Name: Vaca's Creamery
Address: 1436 W Blackhawk St, Chicago, IL
URL: h

Analyze number of Icecream parlours in the area and if they are less than Icecream can also be one division that I can setup in my restaurant

Query 10: Lastly lets fetch some business units across and their websites to further analyze and study

In [20]:
# First Import all required libraries to fetch and connect
import requests
import json

# Step 1: Fetch and enter the generated Yelp API Key (unique to each user)
api_key = '4bu_Cwth58wWv_1EItA96o4C7UXgyuq50aU8O6IgIu6fqW2fYPtwRYYpeyXReB5bUIDdm4XRIzOuacqB3JyvNYtjQ6JQ4ZLeB9LJ6EQ0U3ZOXJN2BDEycz1PqVo9Z3Yx'

# Step 2: Enter Yelp Graph QL API URL
url = 'https://api.yelp.com/v3/graphql'

# Step 3: Define the header and authorizations
headers = {
    'Authorization': f'Bearer {api_key}',
    'Content-Type': 'application/graphql',
}

# 1.Define parameters/data for the query
query = """
{
  search(term: "business", location: "Chicago", limit: 10) {
    business {
      name
      url
    }
  }
}
"""

# 2.Send POST Request with the GraphQL query as data
response = requests.post(url, headers=headers, data=query, timeout=10)

# 3. Check Response Status and Parse Data
if response.status_code == 200:
    data = response.json()
    businesses = data.get('data', {}).get('search', {}).get('business', [])

    if businesses:
        print("Business Units and their URLs in Chicago:")
        for business in businesses:
            print(f"Business Name: {business['name']}")
            print(f"URL: {business['url']}")
            print("-" * 50)
    else:
        print("No businesses found.")
else:
    print(f"Error: {response.status_code}")


Business Units and their URLs in Chicago:
Business Name: O'Brien the Cleaner
URL: https://www.yelp.com/biz/o-brien-the-cleaner-berwyn?adjust_creative=Ceoz5A7JyHPxGhjIOewJ3A&utm_campaign=yelp_api_v3&utm_medium=api_v3_graphql&utm_source=Ceoz5A7JyHPxGhjIOewJ3A
--------------------------------------------------
Business Name: Virgo Web Design
URL: https://www.yelp.com/biz/virgo-web-design-chicago?adjust_creative=Ceoz5A7JyHPxGhjIOewJ3A&utm_campaign=yelp_api_v3&utm_medium=api_v3_graphql&utm_source=Ceoz5A7JyHPxGhjIOewJ3A
--------------------------------------------------
Business Name: 222 Digital Marketing Agency Chicago
URL: https://www.yelp.com/biz/222-digital-marketing-agency-chicago-chicago-5?adjust_creative=Ceoz5A7JyHPxGhjIOewJ3A&utm_campaign=yelp_api_v3&utm_medium=api_v3_graphql&utm_source=Ceoz5A7JyHPxGhjIOewJ3A
--------------------------------------------------
Business Name: Beard Papa’s
URL: https://www.yelp.com/biz/beard-papa-s-chicago?adjust_creative=Ceoz5A7JyHPxGhjIOewJ3A&utm_cam

This will give us deep insights into business verticals and how they create their web portals to share information and market them selves in this competitive world

FINAL CONCLUSION AND INSIGHTS FROM THE ANALYSIS:


Exploring Opportunities for a New Restaurant Using Yelp GraphQL AP:
I
As part of my Enterprise Database Management course, I worked on exploring opportunities to set up a new restaurant in Chicago using the Yelp GraphQL API.
  
The Yelp GraphQL API provided me access to detailed data on restaurants, reviews, locations, and customer preferences. By analysing this data, I aimed to identify key trends, gaps in the market, and potential concepts to consider for a new restaurant
Some insights:


1. Chicago shows a diverse range of cuisines, suggesting there's room for
fusion concepts  
2. Location decision should be based on the concentration of similar restaurants
3. High concentration of pizza restaurants indicates a saturated market
4. Consider areas with fewer competing restaurants but high foot traffic
5. Food events indicate areas with active food culture
6. Businesses with strong web presence perform better

Based on this analysis, I believe there’s an opportunity to introduce a Korean fusion restaurant in Chicago, focusing on: Korean dishes, some fusion dishes and Unique dessert offerings also.
This project gave me a deeper understanding of how to contextualize data within a business framework. It reinforced the importance of identifying market needs, recognizing competition, and tailoring a concept to meet customer needs.
